# Γραμμική Παλινδρόμηση με το dataset House Prices (Kaggle)

## 1. Θεωρία της Γραμμικής Παλινδρόμησης και Πολυωνυμικής Παλινδρόμησης

### Γραμμική παλινδρόμηση

Το απλό γραμμικό μοντέλο περιγράφει γραμμική σχέση μεταξύ της μεταβλητής-στόχου $y$ 
και των χαρακτηριστικών $(x_1,\dots,x_p)$:

$$
\hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \cdots + \beta_p x_p
$$

Οι συντελεστές $\boldsymbol{\beta}$ επιλέγονται ώστε να ελαχιστοποιούν το μέσο τετραγωνικό σφάλμα (MSE):

$$
\min_{\boldsymbol{\beta}} \frac{1}{n} \sum_{i=1}^{n} \big(y_i - \hat{y}_i\big)^2
$$

Η κλειστή μορφή λύσης (όταν $X^\top X$ είναι αντιστρέψιμο) είναι:

$$
\boldsymbol{\beta} = (X^\top X)^{-1} X^\top y
$$

### Πολυωνυμική παλινδρόμηση

Η πολυωνυμική παλινδρόμηση επεκτείνει το γραμμικό μοντέλο εισάγοντας όρους δυνάμεων και αλληλεπιδράσεων των χαρακτηριστικών.

**Παράδειγμα: βαθμός d=2** (δευτερόβαθμιο πολυώνυμο)

Για ένα χαρακτηριστικό $x_1$:

$$
\hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_1^2
$$

Για δύο χαρακτηριστικά $x_1, x_2$:

$$
\hat{y} = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_{11} x_1^2 + \beta_{22} x_2^2 + \beta_{12} x_1 x_2
$$

**Γενικός ορισμός: βαθμός d**

Η πολυωνυμική παλινδρόμηση βαθμού $d$ δημιουργεί νέα χαρακτηριστικά από τις δυνάμεις και γινόμενα των αρχικών μεταβλητών μέχρι βαθμό $d$:

$$
\Phi_d(\mathbf{x}) = \begin{pmatrix} 1 \\ x_1, x_2, \ldots, x_p \\ x_1^2, x_1 x_2, \ldots, x_p^2 \\ \vdots \\ x_1^d, x_1^{d-1}x_2, \ldots, x_p^d \end{pmatrix}
$$

Το μοντέλο παραμένει **γραμμικό ως προς τους συντελεστές** $\boldsymbol{\beta}$:

$$
\hat{y} = \boldsymbol{\beta}^\top \Phi_d(\mathbf{x})
$$

Αυτό σημαίνει ότι μπορούμε να εφαρμόσουμε τα ίδια εργαλεία (OLS, Ridge, Lasso) πάνω στο μετασχηματισμένο σύνολο χαρακτηριστικών.

**Προσοχή:**
- Καθώς αυξάνεται ο βαθμός $d$, αυξάνεται ο αριθμός των χαρακτηριστικών (exponentially) → κίνδυνος **overfitting**.
- Απαιτείται **scaling** (StandardScaler) γιατί τα νέα χαρακτηριστικά έχουν διαφορετικές κλίμακες.
- Συνήθως χρησιμοποιούμε **regularization** (Ridge/Lasso) για να ελέγξουμε το overfitting.
- Η επιλογή του βαθμού γίνεται με **cross-validation** ή σύγκριση train/test metrics.

Σημείωση: Στην πράξη, χρησιμοποιούμε `PolynomialFeatures` (scikit-learn) για τον μετασχηματισμό και `Pipeline` για να συνδυάσουμε scaling + μοντέλο.


## 2. Μετρικές αξιολόγησης (MAE, RMSE, \(R^2\))

- **MAE (Mean Absolute Error):**
$$
MAE = \frac{1}{n} \sum_{i=1}^{n} \big| y_i - \hat{y}_i \big|
$$

- **RMSE (Root Mean Squared Error):**
$$
RMSE = \sqrt{\frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2}
$$

- **\(R^2\) (Συντελεστής Προσδιορισμού):**
$$
R^2 = 1 - \frac{\sum_{i=1}^{n}(y_i - \hat{y}_i)^2}{\sum_{i=1}^{n}(y_i - \bar{y})^2}
$$


In [ ]:
# ΠΑΚΕΤΑ ΠΟΥ ΘΑ ΧΡΗΣΙΜΟΠΟΙΗΣΟΥΜΕ

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split


## 3. Φόρτωση δεδομένων από το CSV

Υποθέτουμε ότι έχετε κατεβάσει το dataset από το Kaggle και ότι το αρχείο
`train.csv` έχει μετονομαστεί σε `house_prices_train.csv` και βρίσκεται στον φάκελο:

`data/house_prices_train.csv`


In [ ]:
ROOT = Path('..')
DATA_PATH = ROOT / 'data' / 'house_prices_train.csv'

print('DATA_PATH:', DATA_PATH)

df = pd.read_csv(DATA_PATH)
df.head()


## 4. Επιλογή χαρακτηριστικών και μεταβλητής-στόχου

Θα κρατήσουμε μόνο μερικές αριθμητικές στήλες για απλότητα.


In [ ]:
feature_names = [
    'LotArea',
    'OverallQual',
    'OverallCond',
    'YearBuilt',
    'GrLivArea',
    'BedroomAbvGr',
    'GarageCars',
]
target_name = 'SalePrice'

df_model = df[feature_names + [target_name]].copy()

# Έλεγχος για NaN
df_model.isna().sum()


### 4.1 Διαχείριση ελλιπών τιμών

Αν υπάρχουν NaN, θα τα αντικαταστήσουμε με τη διάμεσο κάθε στήλης.


In [ ]:
df_model = df_model.dropna(subset=[target_name])
df_model[feature_names] = df_model[feature_names].fillna(
    df_model[feature_names].median()
)
df_model.isna().sum()


## 5. Διαχωρισμός σε train και test set


In [ ]:
X = df_model[feature_names]
y = df_model[target_name]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

X_train.shape, X_test.shape


## 6. Εκπαίδευση μοντέλου Γραμμικής Παλινδρόμησης


In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

y_pred_train = lin_reg.predict(X_train)
y_pred_test = lin_reg.predict(X_test)

print('Συντελεστές:')
for name, coef in zip(feature_names, lin_reg.coef_):
    print(f'  {name:15s} -> {coef:.3f}')
print('\nIntercept:', lin_reg.intercept_)


## 7. Αξιολόγηση του μοντέλου


In [ ]:
mae_train = mean_absolute_error(y_train, y_pred_train)
rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
r2_train = r2_score(y_train, y_pred_train)

mae_test = mean_absolute_error(y_test, y_pred_test)
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
r2_test = r2_score(y_test, y_pred_test)

print('=== TRAIN SET ===')
print(f'MAE : {mae_train:.2f}')
print(f'RMSE: {rmse_train:.2f}')
print(f'R^2: {r2_train:.4f}')

print('\n=== TEST SET ===')
print(f'MAE : {mae_test:.2f}')
print(f'RMSE: {rmse_test:.2f}')
print(f'R^2: {r2_test:.4f}')

## 6. Διάγραμμα: Πραγματικές vs Προβλεπόμενες τιμές


In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(y_test, y_pred_test, alpha=0.4)

min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
plt.plot([min_val, max_val], [min_val, max_val], linestyle='--')

plt.xlabel('Πραγματική τιμή (SalePrice)')
plt.ylabel('Προβλεπόμενη τιμή (SalePrice)')
plt.title('House Prices – Πραγματικές vs Προβλεπόμενες (Linear Regression)')
plt.tight_layout()
plt.show()



## 6. Overfitting και Underfitting

- **Underfitting (υποπροσαρμογή):** το μοντέλο είναι πολύ απλό → 
μεγάλο σφάλμα σε **train** και **test**.
- **Overfitting (υπερπροσαρμογή):** το μοντέλο "αποστηθίζει" το train set → 
πολύ μικρό σφάλμα στο **train**, μεγαλύτερο στο **test**.

Ένας πρακτικός κανόνας:
- Αν $ RMSE_{\text{train}} \ll RMSE_{\text{test}} $ ⇒ πιθανό **overfitting**.
- Αν $ RMSE_{\text{train}} \approx RMSE_{\text{test}} $ αλλά και τα δύο μεγάλα ⇒ **underfitting**.


In [ ]:

# Έλεγχος overfitting σε ένα βήμα (απομονωμένα από τα προηγούμενα κελιά)
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Διαδρομή δεδομένων (ίδια με τα υπόλοιπα κελιά)
ROOT = Path("..")
DATA_PATH = ROOT / "data" / "house_prices_train.csv"

# Φόρτωση
df = pd.read_csv(DATA_PATH)

feature_names = [
    "LotArea",
    "OverallQual",
    "OverallCond",
    "YearBuilt",
    "GrLivArea",
    "BedroomAbvGr",
    "GarageCars",
]
target_name = "SalePrice"

df_model = df[feature_names + [target_name]].copy()
df_model = df_model.dropna(subset=[target_name])
df_model[feature_names] = df_model[feature_names].fillna(df_model[feature_names].median())

X = df_model[feature_names]
y = df_model[target_name]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Μοντέλο
lin = LinearRegression().fit(X_train, y_train)
y_pred_train = lin.predict(X_train)
y_pred_test  = lin.predict(X_test)

rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
rmse_test  = np.sqrt(mean_squared_error(y_test,  y_pred_test))

print(f"RMSE (train): {rmse_train:.2f}")
print(f"RMSE (test) : {rmse_test:.2f}")

# Απλός κανόνας απόφασης
if rmse_train < 0.8 * rmse_test:
    print("⚠️ Πιθανό overfitting (πολύ χαμηλό σφάλμα στο train σε σχέση με το test).")
elif rmse_train > 1.2 * rmse_test:
    print("⚠️ Πιθανό underfitting (το train σφάλμα είναι πολύ μεγάλο σε σχέση με το test).")
else:
    print("✅ Ισορροπία γενίκευσης (train/test σφάλματα είναι κοντά).")


## 8. Σύγκριση Πολυωνυμικών Βαθμών (0-3)

Εδώ συγκρίνουμε την απόδοση πολυωνυμικών μοντέλων βαθμών 0, 1, 2 και 3.
- Βαθμός 0: μόνο intercept (baseline)
- Βαθμός 1: γραμμικό μοντέλο (Linear Regression)
- Βαθμός 2: τετραγωνικά όρια χαρακτηριστικών
- Βαθμός 3: κυβικά χαρακτηριστικά

Θα δούμε πώς αλλάζουν τα metrics (MAE, RMSE, R²) στο train και test set, καθώς και τα plots actual vs predicted.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

# Βεβαιώνουμε ότι έχουμε τα X_train, X_test, y_train, y_test
# (αν δεν τρέξατε τα προηγούμενα κελιά, τρέξτε αυτό το προπαρασκευαστικό κελί)
if 'X_train' not in locals():
    from pathlib import Path
    import pandas as pd
    import numpy as np
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    
    ROOT = Path("..")
    DATA_PATH = ROOT / "data" / "house_prices_train.csv"
    df = pd.read_csv(DATA_PATH)
    
    feature_names = ["LotArea", "OverallQual", "OverallCond", "YearBuilt", "GrLivArea", "BedroomAbvGr", "GarageCars"]
    target_name = "SalePrice"
    
    df_model = df[feature_names + [target_name]].copy()
    df_model = df_model.dropna(subset=[target_name])
    df_model[feature_names] = df_model[feature_names].fillna(df_model[feature_names].median())
    
    X = df_model[feature_names]
    y = df_model[target_name]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Σύγκριση πολυωνυμικών βαθμών
degrees = [0, 1, 2, 3]
results = []

for degree in degrees:
    # Δημιουργία pipeline: PolynomialFeatures + Scaling + LinearRegression
    if degree == 0:
        # Μόνο intercept: απλώς ένα μοντέλο που προβλέπει τη μέση τιμή
        pipeline = Pipeline([
            ('scaler', StandardScaler()),
            ('model', LinearRegression())
        ])
        n_features = 1  # μόνο intercept
    else:
        pipeline = Pipeline([
            ('poly', PolynomialFeatures(degree=degree, include_bias=False)),
            ('scaler', StandardScaler()),
            ('model', LinearRegression())
        ])
        n_features = None  # θα υπολογιστεί μετά τη fit
    
    # Εκπαίδευση
    pipeline.fit(X_train, y_train)
    
    # Προβλέψεις
    y_pred_train = pipeline.predict(X_train)
    y_pred_test = pipeline.predict(X_test)
    
    # Μετρικές
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_pred_train))
    rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)
    
    # Αριθμός χαρακτηριστικών (με τα πολυώνυμα)
    if degree == 0:
        n_features = 1  # μόνο intercept
    else:
        n_features = pipeline.named_steps['poly'].n_output_features_
    
    results.append({
        'Degree': degree,
        'N_features': n_features,
        'MAE_train': mae_train,
        'MAE_test': mae_test,
        'RMSE_train': rmse_train,
        'RMSE_test': rmse_test,
        'R2_train': r2_train,
        'R2_test': r2_test,
        'y_pred_train': y_pred_train,
        'y_pred_test': y_pred_test,
    })
    
    print(f"\n{'='*60}")
    print(f"Βαθμός: {degree} | Χαρακτηριστικά: {n_features}")
    print(f"{'='*60}")
    print(f"TRAIN: MAE={mae_train:.0f}, RMSE={rmse_train:.0f}, R²={r2_train:.4f}")
    print(f"TEST:  MAE={mae_test:.0f}, RMSE={rmse_test:.0f}, R²={r2_test:.4f}")
    print(f"RMSE gap (test - train): {rmse_test - rmse_train:.0f}")

# Δημιουργία σύνοψης DataFrame
results_df = pd.DataFrame([
    {
        'Degree': r['Degree'],
        'N_features': r['N_features'],
        'MAE_train': r['MAE_train'],
        'MAE_test': r['MAE_test'],
        'RMSE_train': r['RMSE_train'],
        'RMSE_test': r['RMSE_test'],
        'R2_train': r['R2_train'],
        'R2_test': r['R2_test'],
    }
    for r in results
])

print("\n" + "="*60)
print("ΣΎΝΟΨΗ")
print("="*60)
display(results_df)

### Plots: Actual vs Predicted για κάθε βαθμό

In [ ]:
# Δημιουργία 2x2 grid διαγραμμάτων (actual vs predicted)
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, (degree, res) in enumerate(zip(degrees, results)):
    ax = axes[idx]
    
    y_pred = res['y_pred_test']
    
    # Scatter plot
    ax.scatter(y_test, y_pred, alpha=0.5, s=30)
    
    # Ideal line (y = x)
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Ideal')
    
    # Labels and title
    ax.set_xlabel('Actual SalePrice', fontsize=11)
    ax.set_ylabel('Predicted SalePrice', fontsize=11)
    ax.set_title(f'Degree {degree} (R² test = {res["R2_test"]:.4f})', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend()

plt.tight_layout()
plt.show()

### Γραφήματα Metrics vs Degree

Παρακάτω εμφανίζουμε πώς αλλάζουν τα RMSE, MAE και R² καθώς αυξάνουμε τον βαθμό του πολυωνύμου.

In [ ]:
# Γραφήματα RMSE, MAE, R²
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# RMSE
axes[0].plot(results_df['Degree'], results_df['RMSE_train'], marker='o', label='Train RMSE', linewidth=2)
axes[0].plot(results_df['Degree'], results_df['RMSE_test'], marker='s', label='Test RMSE', linewidth=2)
axes[0].set_xlabel('Polynomial Degree')
axes[0].set_ylabel('RMSE')
axes[0].set_title('RMSE vs Degree')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(degrees)

# MAE
axes[1].plot(results_df['Degree'], results_df['MAE_train'], marker='o', label='Train MAE', linewidth=2)
axes[1].plot(results_df['Degree'], results_df['MAE_test'], marker='s', label='Test MAE', linewidth=2)
axes[1].set_xlabel('Polynomial Degree')
axes[1].set_ylabel('MAE')
axes[1].set_title('MAE vs Degree')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_xticks(degrees)

# R²
axes[2].plot(results_df['Degree'], results_df['R2_train'], marker='o', label='Train R²', linewidth=2)
axes[2].plot(results_df['Degree'], results_df['R2_test'], marker='s', label='Test R²', linewidth=2)
axes[2].set_xlabel('Polynomial Degree')
axes[2].set_ylabel('R²')
axes[2].set_title('R² vs Degree')
axes[2].legend()
axes[2].grid(True, alpha=0.3)
axes[2].set_xticks(degrees)

plt.tight_layout()
plt.show()

### Scatter Plots (Actual vs Predicted) για κάθε Degree

In [ ]:
# Scatter plots (Actual vs Predicted) για κάθε degree - χωρίς residuals
for degree, res in zip(degrees, results):
    fig, ax = plt.subplots(figsize=(8, 6))
    
    y_pred = res['y_pred_test']
    
    # Scatter plot
    ax.scatter(y_test, y_pred, alpha=0.5, s=40, color='steelblue')
    
    # Perfect prediction line
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Perfect prediction')
    
    ax.set_xlabel('Actual SalePrice', fontsize=12)
    ax.set_ylabel('Predicted SalePrice', fontsize=12)
    ax.set_title(f'Degree {degree}: Actual vs Predicted\nR² = {res["R2_test"]:.4f}, RMSE = {res["RMSE_test"]:.0f}, MAE = {res["MAE_test"]:.0f}', 
                 fontsize=13, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend()
    
    plt.tight_layout()
    plt.show()

### Ερμηνεία των αποτελεσμάτων

**Βαθμός 0 (Baseline - μόνο intercept):**
- Το μοντέλο προβλέπει πάντα τη **μέση τιμή** του training set → πολύ χαμηλή ακρίβεια
- Αναμενόμενα μεγάλα σφάλματα (MAE, RMSE)
- R² θα είναι κοντά στο 0

**Βαθμός 1 (Γραμμικό μοντέλο):**
- Βάση της σύγκρισης
- Ακρίβεια καλή αν υπάρχει γραμμική σχέση μεταξύ χαρακτηριστικών και στόχου

**Βαθμός 2 (Τετραγωνικό):**
- Εμπλοκή τετραγωνικών όρων και αλληλεπιδράσεων
- Συνήθως **καλύτερη** προσαρμογή αν υπάρχουν μη-γραμμικές σχέσεις
- Κίνδυνος αύξησης του overfitting (μεγαλύτερο RMSE gap = test RMSE - train RMSE)

**Βαθμός 3 (Κυβικό):**
- Ακόμη περισσότερη εκφραστικότητα
- **Υψηλό κίνδυνο overfitting**: το μοντέλο μπορεί να "αποστηθίζει" τα δεδομένα
- Αν παρατηρήσετε ότι train R² → 1 αλλά test R² → χαμηλότερη τιμή, τότε έχουμε overfitting

**Σημαντικές παρατηρήσεις:**
1. Κοιτάξτε το **RMSE gap** (test - train): μεγάλο gap υποδεικνύει overfitting
2. Ο βέλτιστος βαθμός είναι αυτός που ισορροπεί train/test metrics
3. Δεν υπάρχει "ένα βέλτιστο" για όλα τα datasets — χρησιμοποιείστε **cross-validation**
4. Υψηλότεροι βαθμοί δημιουργούν **εκθετικά περισσότερα** χαρακτηριστικά → υπολογιστικό κόστος